In [1]:
import pandas as pd
from datasets import load_dataset
from evaluate import load

from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TFAutoModelForSeq2SeqLM

from transformers import create_optimizer, AdamWeightDecay

In [2]:
# load the data
dataset = load_dataset('iwslt2017', 'iwslt2017-zh-en')
data_train = dataset['train']
data_test = dataset['test']
data_val = dataset['validation']

Found cached dataset iwslt2017 (/Users/kangwangkai/.cache/huggingface/datasets/iwslt2017/iwslt2017-zh-en/1.0.0/03ce9110373117c6f6687719f49f269486a8cd49dcad2527993a316cd4b6ad49)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-zh"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("saved_model")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at saved_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [21]:
input_text  = data_test["translation"][0]['en']
tokenized = tokenizer([input_text], return_tensors='tf')
out = model.generate(**tokenized, max_length=128, early_stopping=True)
tokenizer.decode(out[0], skip_special_tokens=True)

'几年前,在TED这里, Peter Skillman提出一个设计挑战, 叫做棉花糖挑战。'

In [25]:
import datasets
import time

In [27]:
ds = datasets.Dataset.from_dict({"en": [pair['en'] for pair in data_test["translation"]], 
                        "zh": [pair['zh'] for pair in data_test["translation"]]})

In [61]:
def translate(data):
    inputs = data['en']
    start = time.time()
    tokenized = tokenizer(inputs, max_length=128, truncation=True, padding=True, return_tensors='tf')
    translated = model.generate(**tokenized, max_length=128, early_stopping=True, num_beams=3)
    with tokenizer.as_target_tokenizer():
        tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    data["translated"] = tgt_text
    end = time.time()
    print('Time used: ', end-start)
    print(tgt_text)
    return data

In [62]:
translated_beam = ds.map(translate, batched=True, batch_size=64)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...bias_layer
......vars
.........0
...layers
......tf_marian_main_layer
.........decoder
............dropout
...............vars
............embed_positions
...............vars
..................0
............embed_tokens
...............vars
..................0
............layers
...............tf_marian_decoder_layer
..................activation_dropout
.....................vars
..................dropout
.....................vars
..................encoder_attn
.....................dropout
........................vars
.....................k_proj
........................vars
...........................0
...........................1
.....................out_proj
........................vars
...........................0
...........................1
.....................q_proj
........................vars
...........................0
...........................1
.....................v_proj
........................vars
.....

Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-04-02 13:33:43         2507
metadata.json                                  2023-04-02 13:33:43           64
variables.h5                                   2023-04-02 13:33:44    312518164


Map:   0%|          | 0/8549 [00:00<?, ? examples/s]

Time used:  114.01722383499146
['几年前,在TED这里, Peter Skillman提出一个设计挑战, 叫做棉花糖挑战。', '这个想法很简单:四人团队 需要从20根意粉中 建造最高的独立结构, 一个带子的院子, 一个绳子的院子和一个棉花糖。', '棉花糖必须在上面。', '尽管这看起来很简单, 但事实上这很难, 因为它迫使人们很快地合作。', '所以我觉得这是一个很有趣的想法, 我把它融入了一个设计工作坊。', '这是一个巨大的成功。', '从那时起,我在世界各地举办了大约70个设计研讨会, 学生、设计师、建筑师, 甚至财富50强的计算机技工, 这项工作中有些东西 揭示了合作本质的深刻教训, 我想与大家分享其中的一些。', '所以,通常,大部分人 都从把自己引向任务开始。', '他们谈论它,他们弄清楚它会是什么样子, 他们为权利而骑马。', '然后他们花一些时间 计划、组织、草图、意粉。', '他们花大部分时间 把棍子组装成不断成长的结构。', '最后,当他们的时间快用完了, 有人拿出棉花糖, 然后他们慢慢地把它放在上面, 然后他们退后,然后 --  ta-da! -- 他们欣赏他们的工作。', '但实际上,大多数时候, 棉花糖的重量 使整个结构被折叠和坍缩。', '所以有很多人 比其他人更有“啊啊啊啊”的时刻, 最糟糕的是,最近刚从商学院毕业的人。', '他们撒谎,作弊,分心, 制造非常烂的结构。', '当然,还有一些团队 拥有更多的“ta-da”结构, 最优秀的团队是最近刚毕业的幼儿园毕业生。', '非常神奇。', '正如Peter告诉我们的, 他们不仅造出了最高的建筑, 而且是其中最有趣的结构。', '所以你想问的问题是: 为什么?为什么?他们有什么问题?', 'Peter喜欢说,没有一个孩子花时间 尝试成为意大利面团公司的CEO,对吧?', '他们不会花时间为权力操控。', '但还有另外一个原因。', '原因就是商业学生 被训练寻找一个正确的计划,对吧?', '然后他们执行它。', '然后,当他们把棉花糖放到顶部时, 他们的时间就快用完了。', '这是一个危机。', '听起来很熟悉吧?对。', '幼儿园的人所做的不同是 他们从棉花糖开始, 他们建造了原型,连续的原型, 总是把棉花糖放在顶部, 所以他们要花很多时间 来修复它

Time used:  128.4949631690979
['音乐是医学。音乐改变我们。', '对Nathaniel来说,音乐是理智的。', '因为音乐允许他 将他的思想和错觉, 并通过他的想象力和创造力, 塑造他的思想和错觉, 变成现实。', '这是从他受苦受难的状态中逃脱出来的。', '我知道这是艺术的本质。', '这正是我们创作音乐的原因, 我们从我们最基本的核心,我们的情感中, 通过我们的艺术透镜,通过我们的创造力, 我们可以将这些情感塑造成现实。', '这种表达的现实 触及我们所有人, 感动我们,启发我们,团结我们。', '对于Nathaniel,音乐让他回到朋友的圈子里。', '音乐的红色力量 将他带回到了音乐家的家庭中, 他们了解他, 认识他的才华并尊敬他。', '我会一直和Nathaniel一起创作音乐, 无论是在迪斯尼音乐厅还是在Skid Row, 因为他提醒了我为什么成为音乐家。', '谢谢。', 'Bruno Giussani:谢谢。谢谢。', '罗伯特·古普塔', '罗伯特·古普塔:我要播放一些我从大提琴家那里偷来的。', '请原谅我。', '我叫简·麦戈尼加尔,我是游戏设计师。', '我已经在网络上制作游戏10年了, 我的目标就是让在现实生活中拯救世界 和在网络游戏中拯救世界一样容易。', '现在,我有一个这个计划, 需要更多人,包括你们, 花更多的时间去玩更大更好的游戏。', '现在,我们每周花30亿小时在网上玩游戏。', '你们中有些人可能会想, “那是花很多时间去玩游戏。', '也许时间太长了, 想想现实世界中我们需要解决多少紧迫的问题。”', '但实际上,根据我对未来研究所的研究, 事实恰恰相反。', '每周30亿小时的游戏 几乎不足以解决世界上最紧迫的问题。', '事实上,我相信如果我们想要在这个星球上度过下一个世纪, 我们需要大幅度地提高这个数字。', '我计算了我们每周需要210亿个小时的游戏。', '所以,这可能是个反直觉的想法, 所以,我再说一遍,让它沉浸其中: 如果我们想要解决像饥饿、贫穷、气候变化、 全球冲突、肥胖这样的问题, 我相信我们需要渴望 每周至少210亿个小时的在线游戏, 在下一个十年结束前。', '不,我是认真的,我是的。', '这就是原因。', '这张照片非常概括了 为什么我认为游戏对于 人类未来的生存

...

Time used:  168.57517790794373
['这会提醒你,你会想介绍你的朋友爱德·库克。', '然后你会看到一幅布瑞尼·斯皮尔斯的照片, 让你想起你想说的这个有趣的故事。', '然后你走进你的厨房, 你将要谈的第四个主题 就是这个奇怪的旅程, 你持续了一年, 你会有一些朋友帮助你记住这个。', '这就是罗马人如何记住他们的演讲-- 不是一字不差的词, 只会搞砸你, 而是一个主题。', '事实上,“主题句子”这个词 来自于希腊语“topos”,意思是“地点”。', '这是人们过去用这些空间术语 来思考口头和修辞时的痕迹。', '“第一”这个词,就像“第一印象宫殿”', '我觉得这很神奇,我非常投入其中。', '我参加了更多的记忆比赛, 我有一个想法,我可能会写得更长一些 关于有竞争力的回忆器的亚文化。', '但有一个问题。', '问题是,记忆竞赛 是一个病态的无聊事件。', '确实,就像一群人坐在一起, 使用半科学—— 我的意思是,最戏剧化的是 当有人开始按摩他们的寺庙的时候。', '我是个记者,我需要写些东西。', '我知道在这些人的脑海中 发生了不可思议的事情, 但我无法理解。', '我意识到,如果我要讲这个故事, 我需要用他们自己的双脚走一点点。', '所以我开始每天早上花15到20分钟, 在我和纽约时报坐下来之前, 试着回忆起一些事情。', '也许是一首诗,也可能是我在跳蚤市场买的旧年书的名字。', '我发现这非常有趣。', '我从没想过会这样。', '这很有趣,因为这其实不是训练你的记忆。', '你正在做的是,你试图做得更好, 在做梦的时候, 创造出这些完全荒谬、 狂野、搞笑, 并且希望让人无法忘怀的画面。', '我非常投入其中。', '这是我穿着我标准的抗记忆器的训练装备。', '这是一副耳罩和一副安全护目镜 除了两个小孔, 因为分心是竞争对手记忆器最大的敌人。', '我最后又回到了一年前我所报道的同一个比赛, 有点像一个参与式新闻报导的实验。', '我想,这也许能为我所有的研究 做一个精彩的开场白。', '问题是,这个实验已经很费劲了', '我赢得了比赛——这本不该发生的。', '现在,能记住演讲、电话号码和购物列表是件好事, 但事实上,这其实与重点无关。', '这些只是些小把戏。', '因为它们基于一些非常基本的原理 来研究大脑是如

Time used:  126.39321708679199
['他们自尊心低落。', '他们更容易被周围的人影响 并且他们更容易患上抑郁症。', '我们认为正是出于这些原因 他们冒了更大的风险 比如酒精和药物使用, 崩溃饮食,整容手术, 未经保护的早期性行为, 以及自我伤害。', '对完美的身体的追求 正在给我们的医疗系统施加压力 并且每年花费我们政府数十亿美元', '我们不会从中成长。', '认为自己超重的女性 不论她们是否超重 缺勤率更高', '百分之十七的女性不会出现在工作面试中 当她们对自己的表情不自信的时候。', '想想这对我们的经济有什么影响。', '如果我们能克服这个,那个机会会是什么样子的。', '释放这种潜力符合我们每一个人的利益。', '但是我们如何做到呢?', '那么,单靠对话,只能让你走这么远。', '这本身是不够的。', '如果你想有所作为, 你必须有所作为。', '我们已经知道有三个关键的方法: 第一,我们要培养身体自信。', '我们必须帮助我们的青少年 制定策略来克服图像的压力 建立他们的自尊心。', '现在,好消息是有很多程序可以做。', '坏消息是,他们大部分都不管用。', '我非常震惊地发现 很多用心良好的项目 无意中使情况变得更糟。', '所以我们需要确保 我们的孩子正在接受的项目 不仅具有积极的影响, 而且还有持久的影响。', '研究显示,最好的方案 涉及六个关键方面: 第一个是家庭、朋友和人际关系的影响。', '第二个是媒体和名人文化, 然后如何处理取笑和欺凌, 我们如何竞争和比较谈论外表—— 一些人称其为“人谈”或“脂肪谈”—— 最后,尊重自己和照顾自己的基础。', '这六件事是任何人严肃对待 提供身体自信教育的关键出发点。', '教育很重要, 但是要解决这个问题, 我们每一个人都要站出来, 成为更好的榜样, 来帮助妇女和女孩们 在我们自己的生活中。', '挑战我们圈子里 女性被看待和谈论的现状。', '我们通过他们的发型或胸部大小 来判断我们的政治家的贡献, 或者推断奥运的 决心或成功 都取决于她是不是个外观者。', '我们需要从人们的所作所为, 而不是他们看起来的样子来判断他们。', '我们都可以开始 负责在自己的社交网络上 发布图片和评论。', '我们可以根据人们的努力和行动 而不是他们的外表 来赞美他们。', '

Time used:  128.4677140712738
['或者更本地化的,像TED大会上的观众一样, 我们现在有集体的TED意识吗? 对于这个集体的TED团体来说, 这与我们每个部分的内心电影是截然不同的?', '我不知道这个问题的答案, 但我认为至少应该认真对待这个问题。', '好的,这个全神贯注的视觉,是一个激进的视觉, 我不知道它是正确的。', '我实际上更自信于第一个疯狂的想法, 意识是最基本的, 而不是第二个,它是全球性的。', '我的意思是,这个观点会提出很多问题, 也会产生很多挑战, 比如说,这些小小的意识 是如何与复杂的意识相结合的, 如果我们能回答这些问题, 那么我认为我们将会在 严肃的认知理论的路上 取得良好的进展。', '如果不是,那么,这也许是科学和哲学中 最困难的问题。', '我们不能指望一夜之间就解决它。', '但是我确实认为我们最终会找到答案。', '我认为,理解意识是理解宇宙 和了解我们自身的真正关键。', '它可能只是有了正确的疯狂点子。', '谢谢。', '我在一个很小的小镇长大, 我有一个非常普通的,低调的成长方式。', '我去了学校,和我的朋友在一起, 我和我的妹妹们战斗。', '这很正常。', '在我15岁的时候, 我当地的一位社区成员 来找我的父母, 想为社区成就奖提名我。', '我的父母说,“嗯,这很好, 但有一个明显的问题。', '她并没有真正取得任何成就。” 他们是正确的,你知道。', '我上学了,成绩不错, 我在学校工作后在我妈妈的美发店工作, 我花了很多时间看《吸血鬼屠夫巴菲》和《达森的溪谷》。', '是的,我知道。多么矛盾。', '但是他们是对的,你知道。', '我没有做任何不寻常的事情。', '如果你从等式中排除残疾, 我并没有做任何可以被视为成就的事情。', '多年后,我在墨尔本高中的第二堂课上, 大约20分钟后,我上了11年级的法律课程, 这个男孩举起手说, “嗨,小姐,你什么时候开始说话?”', '我说,“什么演讲?”', '你知道,我谈论诽谤法已经有20分钟了。', '他说,“你知道,就像,你的动机演讲。', '你知道,当人们坐轮椅来学校时, 他们通常会说,比如说,有灵感的东西?”', '“通常是在大大厅里。”', '就在那一刻,我突然意识到: 这个孩子只经历过残疾人, 成为灵感之源。', '我

Time used:  145.28581023216248
['此外,只有很少的人 跳下桥,过日子,可以谈论它, 百分之一到二, 大多数人说, 他们一旦放下铁轨, 他们就知道他们犯了一个错误, 他们想活下去。', '我告诉人们,这座桥不仅连接了马林和旧金山, 也连接了人们。', '这种联系,或者说我们建造的桥梁, 是我们每一个人都应该努力去做的。', '自杀是可以预防的。', '有帮助,有希望。', '谢谢大家。', '这个世界使你成为你所不是的事物, 但你知道你是什么, 这个问题在你的内心深处燃烧: 你如何变成那样?', '我可能有点独一无二, 但我并不孤单,一点也不孤单。', '所以当我成为时尚模型时, 我觉得我终于实现了 我从小就一直渴望的梦想。', '我的外表最终与我的内心相匹配, 我的内心。', '由于一些复杂的原因,我待会儿会说, 当我看着这张照片, 那时我觉得,吉娜,你成功了, 你成功了,你到了。', '但是去年十月, 我意识到我才刚刚开始。', '我们所有人都被家庭、 宗教、社会、历史时刻, 甚至我们的身体所束缚。', '有些人有勇气打破自由, 不去接受他们肤色的限制, 不去接受周围人的信仰。', '这些人总是威胁现状, 威胁那些被认为可以被接受的东西。', '以我为例,在过去的九年里, 我的一些邻居, 一些我的朋友,同事, 甚至我的经纪人, 并不知道我的历史。', '我认为,在神秘中,这被称为启示。', '这是我的。', '我出生时被分配为男孩 是基于我生殖器的外表。', '我记得当我五岁的时候,在菲律宾, 走在我们的房子里, 我总是在头上穿T恤。', '我妈妈问我,“你怎么总是把T恤挂在头上?”', '我说,“妈妈,这是我的头发,我是个女孩儿。”', '我那时就知道如何自我认同。', '性别一直被认为是一个事实,不可改变, 但现在我们知道它其实更流畅, 更复杂,更神秘。', '由于我的成功,我从来没有勇气 分享我的故事, 不是因为我想错了, 而是因为世界如何对待 那些想要自由的人。', '每一天,我都非常感激 因为我是一个女人。', '我有一个母亲,父亲和家人, 接受我自己。', '很多人并不幸运。', '亚洲文化中有一个悠久的传统 来颂扬性别的奥秘。', '这里有一个虔诚的佛教女神。', '有一个印度女神,Hijra女神。', '我八岁的时候

Time used:  128.3350019454956
['作为一种工具,它可能是不可抗拒的 来增强我们的思维和沟通能力。', '事实上,同样的工具 可能会被证明可以治愈 老年痴呆症和类似疾病。', '我们别无选择,只能打开这扇门。', '不管怎样,选一年 -- 五年或十五年会发生吗?', '很难想象它需要更长的时间。', '我们需要学习如何一起迈出这个步伐。', '谢谢。', '今晚我要跟大家谈谈 从衣橱里出来, 而不是传统意义上的, 不仅仅是同性恋衣橱。', '我想我们都有衣橱。', '你的衣橱可能第一次告诉你爱她的人, 或告诉你怀孕了, 或告诉你得了癌症, 或说我们一生中有过的 任何艰难的谈话。', '所有衣橱都是很艰难的谈话, 尽管我们的主题可能有很大的不同, 进入和走出衣橱的经历是普遍的。', '这太可怕了,我们讨厌它,而且它需要完成。', '几年前,我在南方沃尔纳特咖啡馆工作, 那是一个当地的餐馆, 在那段时间里,我将经历 激进女同性恋的激烈程度: 不刮我的肩膀, 引用Ani DiFranco的歌词作为福音。', '取决于我的短裤的袋状, 以及最近我剃了头发, 这个问题常常会浮现在我身上, 通常是由一个小男孩说: “嗯,你是男孩还是女孩?”', '桌子上会有尴尬的沉默。', '我会更紧地收紧我的下巴, 用更多的报复来握住我的咖啡壶。', '父亲会尴尬地洗掉他的报纸, 母亲会冷眼看着她的孩子。', '但我什么也不说,我看着里面。', '以至于每次我走到桌子前, 一个三到十岁的小孩, 我都准备好去战斗了。', '这感觉很糟。', '所以我答应自己,下次,我会说几句话。', '我会经历艰难的谈话。', '所以在几周内,它又发生了。', '“你是男孩还是女孩?”', '熟悉的沉默,但是这次我已经准备好了, 我准备把所有女性研究101放在桌子上。我有贝蒂·弗里登的名言。', '我引用了葛罗丽亚·斯坦恩的话。', '我甚至从《阴道独白》中得到了这些, 我将要做的。', '我深吸一口气,向下看, 盯着我看, 是一个穿着粉色裙子的四岁女孩, 不是对女权主义决斗的挑战, 只是一个孩子问: “你是男孩还是女孩?”', '于是我再一次深吸一口气,蹲到她旁边, 说,“嘿,我知道这有点困惑。', '我的头发短得像男孩的, 我穿的是男孩的衣服, 但我是个女孩, 你知道有时候你多

In [34]:
translated.save_to_disk("test.hf")

Saving the dataset (0/1 shards):   0%|          | 0/8549 [00:00<?, ? examples/s]

In [42]:
translated[15]

{'en': 'And of course there are teams  that have a lot more "ta-da" structures,  and among the best are recent graduates of kindergarten.',
 'zh': '当然也有一些团队 他们创造了更多的“哈哈”时刻 其中最好的，就是新毕业的幼儿园小朋友',
 'translated': '当然,还有一些团队 拥有更多的“ta-da”结构, 最优秀的团队是最近刚毕业的幼儿园毕业生。'}

In [43]:
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
import jieba

In [65]:
smoothie = SmoothingFunction().method4
ref = jieba.lcut(translated[-4]['zh'])
hyp = jieba.lcut(translated[-4]['translated'])
bleu([ref], hyp, smoothing_function=smoothie)

0.5452469119630863

In [51]:
import pandas as pd

In [52]:
df = pd.DataFrame(translated)

In [63]:
df_beam = pd.DataFrame(translated_beam)

In [57]:
df.to_csv('translated_test_sentences.csv', index=False)

In [64]:
df_beam.to_csv('translated_test_sentences_beam.csv', index=False)